In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

c:\users\abala\appdata\local\programs\python\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\abala\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\users\abala\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('preprocessedDataset.csv')
df.head()
df = df.dropna()
df['speaker'] = df['speaker'].astype('str')
df['response'] = df['response'].astype('str')

In [3]:
def createSpeakerSet(text):
    global promptSet
    global promptDict
    
    text = str(text)
    for word in text.split(' '):
        if word in promptDict:
            promptDict[word] = promptDict[word] + 1
            if promptDict[word] > 4:
                promptSet.add(word)
        else:
            promptDict[word] = 1
        
def createResponseSet(text):
    global respnseSet
    global reponseDict
    
    text = str(text)
    for word in text.split(' '):
        if word in responseDict:
            responseDict[word] = responseDict[word] + 1
            if responseDict[word] > 4:
                responseSet.add(word)
        else:
            responseDict[word] = 1

In [4]:
promptDict = {}
responseDict = {}

promptSet = set()
responseSet = set()

## add the end of the line word
promptSet.add("<END>")
responseSet.add("<END>")

## add the word not in vocab word
promptSet.add("<OTHER>")
responseSet.add("<OTHER>")


## create a set of the vocab for the speaker and the response
_ = df['speaker'].apply(createSpeakerSet)
_ = df['response'].apply(createResponseSet)

## create the sorted vocabulary
promptVocab = sorted(list(promptSet))
responseVocab = sorted(list(promptSet))

## get the size of the vocab
promptVocabSize = len(promptVocab)
responseVocabSize = len(responseVocab)

## get the longest prompt and response
#maxLenPrompt = max([len(str(txt)) for txt in list(df['speaker'])])
maxLenPrompt = 50 + 1
#maxLenResponse = max([len(str(txt)) for txt in list(df['response'])])
maxLenResponse = 50 + 1

## create the empty input and output vectors
encoderInputSize = (len(df), maxLenPrompt, promptVocabSize)
encoder_input = np.zeros(encoderInputSize, dtype = "float32")

decoderInputSize = (len(df), maxLenResponse, responseVocabSize)
decoder_input = np.zeros(decoderInputSize, dtype = "float32")

decoder_output = np.zeros(decoderInputSize, dtype = "float32")

## populate the empty input and output vectors
inputText = list(df['speaker'])
outputText = list(df['response'])

for i, (i_t, o_t) in enumerate(zip(inputText, outputText)):
    #print(inputText[i])
    inputList = inputText[i].split(' ')
    outputList = outputText[i].split(' ')
    endInd = len(inputList)
    if endInd > maxLenPrompt - 1:
        endInd = maxLenPrompt - 1  
    for x in range(endInd):
        if inputList[x] in promptVocab:
            ind = promptVocab.index(inputList[x])
        else:
            ind = promptVocab.index("<OTHER>")
        encoder_input[i][x][ind] = 1.0
    encoder_input[i][x+1][promptVocab.index("<END>")] = 1.0
    endInd = len(outputList)
    if endInd > maxLenResponse - 1:
        endInd = maxLenResponse - 1
    for x in range(endInd):
        if outputList[x] in responseVocab:
            ind = responseVocab.index(outputList[x])
        else:
            ind = responseVocab.index("<OTHER>")
        decoder_input[i][x][ind] = 1.0
        if i >=1:
            decoder_output[i][x-1][ind] = 1.0
    decoder_input[i][x+1][responseVocab.index("<END>")] = 1.0
    decoder_output[i][x][responseVocab.index("<END>")] = 1.0

In [8]:
encoder_inputs = keras.Input(shape = (None, promptVocabSize))
encoder = keras.layers.LSTM(256, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape = (None, responseVocabSize))

decoder_lstm = keras.layers.LSTM(256, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = keras.layers.Dense(responseVocabSize, activation = "softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.fit(
    [encoder_input, decoder_input],
    decoder_output,
    batch_size=32,
    epochs=50
    #,    validation_split=0.2,
)

Epoch 1/50
305/305 [==============================] - 560s 2s/step - loss: 1.5820 - accuracy: 0.0521
Epoch 2/50
305/305 [==============================] - 558s 2s/step - loss: 1.5814 - accuracy: 0.0521
Epoch 3/50
305/305 [==============================] - 438s 1s/step - loss: 1.5815 - accuracy: 0.0521
Epoch 4/50
305/305 [==============================] - 404s 1s/step - loss: 1.5813 - accuracy: 0.0521
Epoch 5/50
305/305 [==============================] - 400s 1s/step - loss: 1.5798 - accuracy: 0.0521
Epoch 6/50
305/305 [==============================] - 397s 1s/step - loss: 1.5807 - accuracy: 0.0521
Epoch 7/50
305/305 [==============================] - 403s 1s/step - loss: 1.5967 - accuracy: 0.0519
Epoch 8/50
305/305 [==============================] - 404s 1s/step - loss: 1.5839 - accuracy: 0.0520
Epoch 9/50
305/305 [==============================] - 426s 1s/step - loss: 1.5808 - accuracy: 0.0521
Epoch 10/50
305/305 [==============================] - 410s 1s/step - loss: 1.5814 - accura